In [1]:
#Importing the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,UpSampling2D,BatchNormalization,Reshape

Using TensorFlow backend.


In [2]:
def generator_function( ):
    model = Sequential( )
    model.add(Dense( units =1024 , activation="relu" , input_dim=900) )
    model.add(Dense(2048, activation='relu') )
    #Dense layers in starting
    model.add(Dense(224*8*8 , activation="relu" ) )
    model.add(BatchNormalization())
    model.add(Reshape((8,8,224),input_shape=(224*8*8,)))
    #Reshaping 1_D tensor in form of 3_D tensor
    model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
    model.add(UpSampling2D(size=(2,2)))
    model.add(Conv2D(32,(3,3),padding='valid',activation='relu'))
    model.add(UpSampling2D(size=(2,2)))
    model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
    model.add(UpSampling2D(size=(2,2)))
    model.add(Conv2D(32,(3,3),padding='same',activation='relu'))
    model.add(UpSampling2D(size=(2,2)))
    model.add(Conv2D(16,(5,5),padding='same',activation='relu'))
    model.add(UpSampling2D(size=(2,2)))
    model.add(Conv2D(1,(7,7),padding='same',activation='sigmoid'))
    #convolutional layers and max_pooling layers to make the final output of generator of size of image
    return model


In [3]:
def discriminator_function():
    model=Sequential()
    model.add(Conv2D(16,(7,7),padding='same',activation='relu',input_shape=(224,224,1)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(32,(5,5),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(224,(3,3),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(2048,activation='relu'))
    model.add(Dense(1024,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    return model


In [4]:
def generator_containing_discriminator(generator,discriminator):
    model=Sequential()
    model.add(generator)
    discriminator.trainable=False
    model.add(discriminator)
    return model


In [5]:
#Reading fingerprint images
import os
import cv2
from PIL import Image

def take_input():

    prefix="C:\\Users\\DELL1\\Desktop\\Gan\\DB1\\"
    imageNames=os.listdir(prefix)
    images=[]
    for imageName in imageNames :
        image_path = prefix + imageName
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image = cv2.cvtColor( image , cv2.COLOR_RGB2GRAY)
        image = image.reshape (224 , 224 , 1)
        image = image.astype( 'float32' )/255
        images.append( image )
    return images

In [6]:
from keras.optimizers import Adam

def train(batch_size):
    X_train = take_input()
    discriminator=discriminator_function()
    generator=generator_function()
    discriminator_on_generator_model=generator_containing_discriminator(generator,discriminator)
    d_optim=Adam(lr=0.00001,beta_1=.9,beta_2=.999)
    g_optim=Adam(lr=.0001,beta_1=.9,beta_2=.999)
    generator.compile(loss='binary_crossentropy',optimizer=g_optim)
    discriminator_on_generator_model.compile(loss='binary_crossentropy',optimizer=g_optim)
    discriminator.trainable=True
    discriminator.compile(loss='binary_crossentropy',optimizer=d_optim)
    
    noise=np.zeros((batch_size,900))
    for epoch in range(2):
        print("Epoch is",epoch)
        print("Number of Batches", int(len(X_train)/batch_size))
        for index in range(int(len(X_train)/batch_size)):
            for i in range(batch_size):
                noise[i,:]=np.random.uniform(0,1,900)
            noise=noise.reshape(batch_size,900)
            image_batch=X_train[index*batch_size:(index+1)*batch_size]
            generated_images=generator.predict(noise,verbose=0)
            #generated images will contain the images generated by generator, no. of images will be batch_size
            if index%20==0:
                #saving the image after every 20 batches
                for i in range(batch_size):
                    image=generated_images[i]*255
                    image= Image.fromarray(image.reshape(224,224).astype('uint8')).convert('RGB')
                    image.save("C:\\Users\\DELL1\\Desktop\\Gan\\"+str( epoch )+" "+str( index )+".png")
            X=np.concatenate((np.asarray(image_batch),generated_images))
            #Taking real & gen.images in same array
            y=[1]*batch_size + [0]*batch_size
            #Labelling images according to ther class
            d_loss=discriminator.train_on_batch(X,y)
            #This step will update the weights of discriminator model and output the loss after update
            print("batch %d d_loss : %f" %(index,d_loss))
            for i in range(batch_size):
                noise[i,:]=np.random.uniform(0,1,900)
            noise=noise.reshape(batch_size,900)
            #generating noise of batch_size
            discriminator.trainable=False
            noise1=np.random.uniform(0,1,900)
            noise1=noise1.reshape(1,900)
            g_loss= discriminator_on_generator_model.train_on_batch(noise,[1]*batch_size)
            #The combined model has given the noise as input and output of one since we want to generate real images
            ## This step will update the weights of generator model and output the loss after update
            discriminator.trainable=True
            print("batch %d g_loss : %f" %(index,g_loss))
            #Saving both the models after every batch
        generator.save_weights("C:\\Users\\DELL1\\Desktop\\Gan\\gen"+str(epoch)+'.h5',True)
        discriminator.save_weights("C:\\Users\\DELL1\\Desktop\\Gan\\dis"+str(epoch)+'.h5',True)
            
                                                                

In [8]:
def generate(batch_size):
    generator=generator_function()
    generator.compile(loss='binary_crossentropy',optimizer='SGD')
    generator.load_weights("C:\\Users\\DELL1\\Desktop\\Gan\\gen1.h5")
    noise=np.zeros((batch_size,900))
    for i in range(batch_size):
        noise[i,:]=np.random.uniform(0,1,900)
    generated_images= generator.predict(noise,verbose=1)
    for i in range(batch_size):
        image=generated_images[i]*255
        image=Image.fromarray(image.reshape(224,224).astype('uint8')).convert('RGB')
        image.save("C:\\Users\\DELL1\\Desktop\\Gan\\Generated_images\\"+str(i)+".png")
        #saving the generated images

In [7]:
#As I am using my laptop with just 8gb ram, I gave epochs=2. Normally, it should be at least equal to 100 to produce accurate results. 

train=train(64)

Epoch is 0
Number of Batches 5
batch 0 d_loss : 0.689784
batch 0 g_loss : 0.665754
batch 1 d_loss : 0.679770
batch 1 g_loss : 0.642119
batch 2 d_loss : 0.669400
batch 2 g_loss : 0.619376
batch 3 d_loss : 0.663857
batch 3 g_loss : 0.599498
batch 4 d_loss : 0.662420
batch 4 g_loss : 0.581712
Epoch is 1
Number of Batches 5
batch 0 d_loss : 0.649675
batch 0 g_loss : 0.569060
batch 1 d_loss : 0.651766
batch 1 g_loss : 0.559485
batch 2 d_loss : 0.642660
batch 2 g_loss : 0.552598
batch 3 d_loss : 0.642102
batch 3 g_loss : 0.547524
batch 4 d_loss : 0.646556
batch 4 g_loss : 0.543223


In [9]:
gen=generate(64)

64/64 [==============================] - 13s 201ms/step


In [11]:
generator=generator_function()
generator.output_shape

(None, 224, 224, 1)